In [13]:
import dolfinx, gmsh, gmsh_helpers, ufl, time
from mpi4py import MPI
from tqdm import tqdm
import numpy as np

# Physical quantities
tx_freq = 200e3 # Hz
c0 = 1481 # m.s-1
omega = 2*np.pi*tx_freq
k0 = omega / c0 # wavenumber
wave_len = c0 / tx_freq

# Transducer properties
tx_radius = 15e-2
tx_aperture_radius = 15e-2
alpha_aperture = np.arcsin(tx_aperture_radius / (2*tx_radius))
tx_marker, boundary_marker, ac_domain_marker = 1, 2, 3

# Domain parameters
topology_dim = 2
dims = (30e-2, 35e-2)

# Spatial discretisation parameters
degree = 3
n_wave = 3
h_elem = wave_len / n_wave

# Temporal discretisation parameters
dt = 1 / (10 * tx_freq)
T = 2e-4
t_mesh = np.arange(0, T, dt)

In [14]:
''' Mesh generation '''
gmsh.initialize()

model = gmsh.model
model.add("TxDomain")

tx_center = model.geo.addPoint(tx_radius*np.cos(alpha_aperture),0,0, meshSize=h_elem, tag=10)
points = []

points.append(model.geo.addPoint(tx_radius*np.cos(-alpha_aperture+np.pi) + tx_radius*np.cos(alpha_aperture), tx_radius*np.sin(-alpha_aperture+np.pi), 0, meshSize=h_elem, tag=0))
points.append(model.geo.addPoint(tx_radius*np.cos(alpha_aperture+np.pi) + tx_radius*np.cos(alpha_aperture), tx_radius*np.sin(alpha_aperture+np.pi), 0, meshSize=h_elem, tag=1))
rect_domain_geom = [[0., -1/2], [1., -1/2], [1., 1/2], [0., 1/2]]
for rect_geom in rect_domain_geom:
    points.append(model.geo.addPoint(*[gg*dd for gg, dd in zip(rect_geom, dims)],0, meshSize=h_elem, tag=len(points)))

lines = []
lines.append(model.geo.addCircleArc(points[0], tx_center, points[1]))
for pt in points[1:]:
    lines.append(model.geo.addLine(points[pt], points[(pt+1)%6]))

# Curveloop and Surface
curveloop = model.geo.addCurveLoop(lines)
domain = model.geo.addPlaneSurface([curveloop])

# This command is mandatory and synchronize CAD with GMSH Model. The less you launch it, the better it is for performance purpose
gmsh.model.geo.synchronize()

# Physical groups
gmsh.model.addPhysicalGroup(1, [lines[0]], tx_marker)
gmsh.model.setPhysicalName(1, tx_marker, "Tx")
gmsh.model.addPhysicalGroup(1, lines[1:], boundary_marker)
gmsh.model.setPhysicalName(1, boundary_marker, "Domain boundary")
gmsh.model.addPhysicalGroup(2, [domain], ac_domain_marker)
gmsh.model.setPhysicalName(2, ac_domain_marker, "Acoustic domain")

# Mesh
model.mesh.generate(topology_dim)
gmsh.write("TxDomain.msh")

# Gmsh to dolfinx mesh
mesh, cell_tags, facet_tags = gmsh_helpers.gmsh_model_to_mesh(model, cell_data=True, facet_data=True, gdim=topology_dim)

# Finalize GMSH
gmsh.finalize()

In [15]:
P = dolfinx.FunctionSpace(mesh, ("Lagrange", degree))

# Retreive the transducer facet indexes to apply the bc later on
tx_facets = facet_tags.indices[facet_tags.values == tx_marker]
tx_dofs = dolfinx.fem.locate_dofs_topological(P, topology_dim - 1, tx_facets)
p_source = dolfinx.Function(P)

In [16]:
# Initial state of the acoustic domain
p0 = dolfinx.Function(P)
p0.vector.set(0)
p1 = dolfinx.Function(P)
p1.vector.set(0)

In [17]:
# Define variational problem
u = ufl.TrialFunction(P)
v = ufl.TestFunction(P)

a = u*v*ufl.dx + dt*dt*c0*c0*ufl.inner(ufl.grad(u), ufl.grad(v))*ufl.dx
L = 2*p1*v*ufl.dx-p0*v*ufl.dx

In [18]:
xdmf_file = dolfinx.io.XDMFFile(MPI.COMM_WORLD, "tx_propagation_test_lf_deg3.xdmf", "w")
xdmf_file.write_mesh(mesh)

for tt in tqdm(t_mesh):
    p_source.vector.set(np.sin(omega*tt)) 
    tx_bc = dolfinx.DirichletBC(p_source, tx_dofs)

    problem = dolfinx.fem.LinearProblem(a, L, bcs=[tx_bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
    p = problem.solve()

    p.name = "pressure"
    xdmf_file.write_function(p, tt)
    
    with p0.vector.localForm() as p_loc_0, p1.vector.localForm() as p_loc_1:
        p_loc_1.copy(p_loc_0)

    with p1.vector.localForm() as p_loc_1, p.vector.localForm() as p_loc:
        p_loc.copy(p_loc_1)

xdmf_file.close()

100%|██████████| 401/401 [12:53<00:00,  1.93s/it]


![alt text](p_fields.png "p field")